In [ ]:
import zipfile
import os

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/Final_Project/Jun/korean 01 data (3).zip'  # 업로드한 ZIP 파일 이름

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('korean_01_data')  # 'extracted_folder'는 압축 해제할 폴더 이름


In [ ]:
import zipfile
import os

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/Final_Project/Jun/jsonfile (3).zip'  # 업로드한 ZIP 파일 이름

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('jsonfile')  # 'extracted_folder'는 압축 해제할 폴더 이름


In [ ]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
# 이미지 로드
def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img , (256,256))
    img = img / 255.0
    return img

In [ ]:
# json 파일 로드
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # gender = 1 if data['info']['gender'] =='F' else 0
    # skin_type = data['info']['skin_type']
    # sensitive = data['info']['sensitive']

    pigmentation = data['annotations']['forehead_wrinkle']

    return pigmentation

In [ ]:
from sklearn.model_selection import train_test_split

image_dir = r"/content/korean_01_data"
json_dir = r"/content/jsonfile"

images = []
metadata = []

four_or_five_path = []

In [ ]:
for main_folder in os.listdir(image_dir):  # 'digit cam', 'pad', 'cell phone'
    main_folder_path = os.path.join(image_dir, main_folder)
    if os.path.isdir(main_folder_path):  # 메인 폴더인지 확인
        for sub_folder in os.listdir(main_folder_path):  # '0001', '0002', '0003'
            sub_folder_path = os.path.join(main_folder_path, sub_folder)
            if os.path.isdir(sub_folder_path):  # 서브 폴더인지 확인
                for filename in os.listdir(sub_folder_path):  # 이미지 파일들
                    if filename.endswith('.jpg'):  # .jpg 파일만 처리
                        image_path = os.path.join(sub_folder_path, filename)

                        # # 여기서 특정 조건에 맞는 파일만 선택하도록 수정
                        # if 'F' not in filename:  # L15가 포함된 파일만 선택
                        #     continue  # L15가 포함되지 않으면 건너뜀


                        # JSON 파일명 생성: 'cropped_' 제거하고 '.jpg' -> '.json'
                        json_filename = filename.replace('cropped_', '').replace('.jpg', '')

                        # json_dir도 image_dir처럼 동일한 폴더 구조를 반영하여 경로 설정
                        json_folder_path = os.path.join(json_dir, main_folder, sub_folder)
                        json_path = os.path.join(json_folder_path, json_filename)  # JSON 파일 경로


                        # 이미지와 JSON 로드
                        image = load_image(image_path)
                        # if image is None:
                        #     continue  # 이미지 로드 실패시 건너뛰기
                        pigmentation = load_json(json_path)

                        if pigmentation in [1,6]:
                            continue

                        if pigmentation in [0,2,3,4,5]:
                            four_or_five_path.append(json_path)

                            if pigmentation in [0,2]:
                                pigmentation = 0
                            elif pigmentation in [3,4,5]:
                                pigmentation = 1



                        images.append(image)  # images 리스트에 추가
                        metadata.append(pigmentation)  # metadata 리스트에 추가

In [ ]:
import pandas as pd
columns = ['Pigmentation']
metadata_df = pd.DataFrame(metadata, columns =columns)
metadata_df['Pigmentation'].value_counts()

In [ ]:
# 리스트를 numpy 배열로 변환
metadata = np.array(metadata)

# 데이터 준비
X = np.array(images, dtype=np.float32)
y = np.array(metadata, dtype=np.float32)  # 타겟 (pigmentation)


In [ ]:
from sklearn.model_selection import train_test_split

# 이미지와 메타데이터를 함께 train_test_split에 전달
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)




# 결과
X_train_images = X_train
X_val_images = X_val


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.regularizers import l1, l2

# 이미지 입력
image_input = Input(shape=(256, 256, 3), name='image_input')
x = layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(image_input)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64,(3,3), activation='relu', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), activation='swish', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(512,(3,3), activation='relu', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
x = layers.MaxPooling2D((2, 2))(x)

# x = layers.Conv2D(1024, (3, 3), activation='swish',kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Conv2D(2048,(1,1), activation='swish',kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Conv2D(4096,(3,3), activation='swish',kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
# x = layers.MaxPooling2D((2,2))(x)

x = layers.GlobalAveragePooling2D()(x)  # Global Average Pooling
x = layers.Dropout(0.5)(x)


x = layers.Dense(512, activation='swish', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)  # 더 큰 Dense 레이어
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# 출력 레이어
output = layers.Dense(1, activation='sigmoid')(x)  # 이진 분류

# 모델 생성
model = models.Model(inputs=image_input, outputs=output)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)  # 학습률 조정
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, Input
# from tensorflow.keras.regularizers import l2

# # Residual Block 정의
# def residual_block(x, filters):
#     shortcut = x
#     # 채널 수를 맞추기 위해 1x1 컨볼루션 추가 (Projection Shortcut)
#     if x.shape[-1] != filters:
#         shortcut = layers.Conv2D(filters, (1, 1), padding='same', kernel_regularizer=l2(0.01))(shortcut)
#         shortcut = layers.BatchNormalization()(shortcut)

#     # Residual 경로
#     x = layers.Conv2D(filters, (3, 3), padding='same', activation='swish', kernel_regularizer=l2(0.01))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Conv2D(filters, (3, 3), padding='same', activation=None, kernel_regularizer=l2(0.01))(x)
#     x = layers.BatchNormalization()(x)

#     # 입력(shortcut)과 출력(x)을 더하기
#     x = layers.Add()([x, shortcut])
#     x = layers.Activation('swish')(x)
#     return x

# # 이미지 입력
# image_input = Input(shape=(256, 256, 3), name='image_input')

# # 첫 번째 컨볼루션 레이어 (초기 입력 처리)
# x = layers.Conv2D(64, (3, 3), activation='swish', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(image_input)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPooling2D((2, 2))(x)

# # Residual Block 적용
# x = residual_block(x, 64)  # 필터 수 64
# x = residual_block(x, 128)  # 필터 수 128
# x = layers.MaxPooling2D((2, 2))(x)

# x = residual_block(x, 256)  # 필터 수 256
# x = residual_block(x, 512)  # 필터 수 512
# x = layers.MaxPooling2D((2, 2))(x)

# # Global Average Pooling
# x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dropout(0.5)(x)

# # Fully Connected Layer
# x = layers.Dense(1024, activation='swish', kernel_regularizer=l2(0.01), kernel_initializer='he_normal')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)

# # 출력 레이어
# output = layers.Dense(1, activation='sigmoid')(x)  # 이진 분류

# # 모델 생성
# model = models.Model(inputs=image_input, outputs=output)

# # 모델 컴파일
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)  # 학습률 조정
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# # 모델 요약 출력
# model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # val_loss를 모니터링
    patience=20,          # 개선되지 않은 에포크를 3번 참음
    restore_best_weights=True  # 최상의 모델 가중치를 복원
)

In [ ]:
history = model.fit(
    X_train_images,  # X_train_metadata 대신 y_train 사용 (pigmentation만 필요)
    y_train,  # 타겟 라벨
    validation_data=(X_val_images, y_val),  # 검증 데이터도 동일하게 전달
    epochs=300,
    batch_size=64,  # 올려볼것 32 64
    #callbacks=[early_stopping]
)

# 학습이 완료되면 학습 과정에서의 손실과 메트릭스를 확인할 수 있습니다.
print("Training History: ", history.history)


In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(
    {'image_input': X_val_images, 'metadata_input': y_val},  # 검증 데이터에서 metadata_input은 pigmentation만 사용
    y_val  # 검증 데이터의 실제 레이블 (pigmentation 값)
)

print("Test Loss: ", test_loss)
print("Test Accuracy: ", test_accuracy)


# 0.58 0.56
# 3으로 합친 후 : 0.569, stratify = y : 0.538
# 모든 데이터 합친 후 : 0.52

# 0.46
# 0.56 batch normalization
# 0.40 0.0001
# 0.19 0.25  0.54 메타데이터 층 증가
# 0.62(과대적합)
# dropout(0.5) 0.678
# Dense만 Dropout 0.64
# dropout 추가 0.63(과대적합)

# 이진분류 0.668



# swish 0.00001 == 0.665
# 300epoch ==0.7
# 450 epcoh ==0.73
# 400 epoch = 0.68
# 500 epoch = 0.71
# 600 epoch = 0.76
# 700 epoch = 0.79
# 900 epoch = 0.79

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # 첫 번째 그래프 (좌측)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 학습 정확도 (accuracy) 및 검증 정확도 (val_accuracy) 그래프
plt.subplot(1, 2, 2)  # 두 번째 그래프 (우측)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 그래프를 화면에 표시
plt.tight_layout()
plt.show()

In [ ]:
# model.save('model76-1.h5')